In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import scipy.io as sio

from model import *
from preprocess import *

from trainer.multi_gpu_train import Trainer
import cfgs.config_eurlex_with_preprocess as config

run_preprocess = False

In [2]:
path = os.path.join(config.cfg.dir_name, "train_data")
X, Y = data_loader.data_loader(path, config.cfg)
print("Data-Stats:")
print("Num instances = ", X.shape[0])
print("Feature dimensionality = ", X.shape[-1])
print("Label dimensionality = ", Y.shape[-1])
print("Mean pts per label = ", Y.nnz/Y.shape[1])
print("Mean labels per pt = ", Y.nnz/Y.shape[0])

print("Param-stats:")
print("Projection-Dim: %d"%config.cfg.d)
print("Num Projections: %d"%config.cfg.m)

Data-Stats:
Num instances =  15539
Feature dimensionality =  5000
Label dimensionality =  3993
Mean pts per label =  20.6686701728
Mean labels per pt =  5.31115258382
Param-stats:
Projection-Dim: 250
Num Projections: 1000


In [3]:
if run_preprocess:
    D = X.shape[-1]
    d = config.cfg.d
    m = config.cfg.m
    W0, Wx = pca.train_pca(X, d)
    B0 = clustering.train_kmeans(Wx, m, ngpu = 1).T
    Z0 = prototypes.get_prototypes(Y, Wx, B0, num_pts_per_cluster=config.cfg.num_pts_per_cluster)

    path = os.path.join(config.cfg.dir_name, "init_params_faiss.mat")
    sio.savemat(path, {'W':W0, 'B':B0, 'Z':Z0})

In [4]:
m = Trainer(config.cfg).train()

W variable created on cpu
B variable created on cpu
Z variable created on cpu
tower_0/Mean
tower_0/total_loss
tower_1/Mean
tower_1/total_loss
Instructions for updating:
Use the retry module or similar alternatives.


2019-01-10 01:55:17,767:WARNING:From /n/home13/sravindranath/.conda/envs/tf/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:198: retry (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Use the retry module or similar alternatives.
2019-01-10 01:58:52,588:INFO:TRAIN-BATCH Iter = 100, t = 82.36, Loss = 492.72, Prec@1: 0.7422, Prec@3: 0.5924, Prec@5: 0.4828
2019-01-10 01:58:52,790:INFO:VAL-ALL Iter = 100, t = 0.20, Loss = 494.03, Prec@1: 0.7304, Prec@3: 0.5662, Prec@5: 0.4642
2019-01-10 02:00:02,126:INFO:TRAIN-BATCH Iter = 200, t = 150.32, Loss = 485.97, Prec@1: 0.7930, Prec@3: 0.6562, Prec@5: 0.5336
2019-01-10 02:00:02,339:INFO:VAL-ALL Iter = 200, t = 0.21, Loss = 484.32, Prec@1: 0.7548, Prec@3: 0.5853, Prec@5: 0.4817


KeyboardInterrupt: 